In [1]:
import cv2
import json
import requests
import http.client
import urllib.request
import urllib.parse
import urllib.error
import base64

In [2]:
def analyze_frame(image_path, threshold):
    headers = {
        # Request headers
        'Content-Type': 'application/octet-stream',
        'Prediction-key': 'da332d2afd7446509f189200f7b85126',
    }

    params = urllib.parse.urlencode({
        # Request parameters
        'iterationId': '',
        'application': '',
    })
    
    image = open(image_path, "rb").read()

    try:
        conn = http.client.HTTPSConnection('southcentralus.api.cognitive.microsoft.com')
        conn.request("POST", "/customvision/v1.1/Prediction/4e402c59-95bf-4deb-a937-3b8e9bac2676/image?%s" % params, image, headers)
        response = conn.getresponse()
        analysis = json.loads(response.read())
        #print(analysis)
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    
    result = 0
    dict_list = analysis['Predictions']
    for dictionary in dict_list:
        if dictionary['Tag'] == 'not_gun':
            if dictionary['Probability'] >= threshold:
                return 0
        elif dictionary['Probability'] > result:
                result = dictionary['Probability']
    return result

In [3]:
camera = cv2.VideoCapture(0)

In [4]:
#analysis every frame
threshold = 0.75

i = 0
while(True):
    ret, frame = camera.read()
    cv2.imshow('frame', frame)
    i += 1
    
    if i == 150:
        cv2.imwrite('capture.jpg', frame)
        alarm = analyze_frame('capture.jpg', threshold)
        if alarm > threshold:
            cv2.putText(frame, 'GUN', (10,500), cv2.FONT_HERSHEY_SIMPLEX, 4,(255,255,255),2,cv2.LINE_AA)
            print("ALARM: " + str(alarm))
        else:
            print("NO GUN DETECTED: " + str(alarm))
        i = 0
    
    key = cv2.waitKey(3)
    #ESC key press
    if key%256 == 27:
        break
        
camera.release()
cv2.destroyAllWindows()

NO GUN DETECTED: 0.372088462
NO GUN DETECTED: 0.6016325
NO GUN DETECTED: 0.26129362
NO GUN DETECTED: 0
NO GUN DETECTED: 0
NO GUN DETECTED: 0
NO GUN DETECTED: 0
NO GUN DETECTED: 0
NO GUN DETECTED: 0
NO GUN DETECTED: 0.20884414
NO GUN DETECTED: 0
ALARM: 0.9982698
ALARM: 0.9526491
ALARM: 0.953177
ALARM: 0.941794
NO GUN DETECTED: 0
NO GUN DETECTED: 0
NO GUN DETECTED: 0.254703283
NO GUN DETECTED: 0.5483509
NO GUN DETECTED: 0


In [4]:
#analysis on key press
while(True):
    ret, frame = camera.read()
    cv2.imshow('frame', frame)
    
    key = cv2.waitKey(3)
    #ESC key press
    if key%256 == 27:
        break
    #SPACE key press
    elif key%256 == 32:
        cv2.imwrite('capture.jpg', frame)
        alarm = analyze_frame('capture.jpg', 0.75)
        if alarm:
            print("ALARM")
        else:
            print("NO GUN DETECTED")
        
camera.release()
cv2.destroyAllWindows()

NO GUN DETECTED
ALARM
ALARM
NO GUN DETECTED
NO GUN DETECTED
NO GUN DETECTED
ALARM
NO GUN DETECTED
ALARM
ALARM
ALARM
ALARM
ALARM
ALARM


KeyError: 'Predictions'